In [1]:
# Cell 1: Install Required Packages
!pip install -q transformers torch sentence-transformers faiss-cpu PyPDF2 langchain tiktoken datasets accelerate numpy pandas scikit-learn ipywidgets

print("✅ All packages installed successfully!")
print("📋 Installed: transformers, torch, sentence-transformers, faiss, PyPDF2, etc.")
print("🚀 Ready to proceed to Cell 2!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import PyPDF2
import numpy as np
import pandas as pd
from typing import List, Dict, Any
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    AutoModel,
    pipeline,
    T5ForConditionalGeneration,
    T5Tokenizer
)
from sentence_transformers import SentenceTransformer
import faiss
import re
from sklearn.metrics.pairwise import cosine_similarity
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Check system capabilities
print(f"📱 System Information:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"   Using device: {device}")

print("✅ All libraries imported successfully!")
print("🚀 Ready to proceed to Cell 3!")

📱 System Information:
   PyTorch version: 2.6.0+cu124
   CUDA available: False
   Using device: cpu
✅ All libraries imported successfully!
🚀 Ready to proceed to Cell 3!


In [3]:
class DocumentProcessor:
    """Handles loading PDF/TXT files and breaking them into chunks"""

    def __init__(self, chunk_size: int = 300, overlap: int = 50):
        self.chunk_size = chunk_size
        self.overlap = overlap
        print(f"📄 DocumentProcessor ready: {chunk_size} words per chunk")

    def load_pdf(self, file_path: str) -> str:
        """Extract text from PDF file"""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        except Exception as e:
            print(f"❌ Error loading PDF {file_path}: {e}")
            return ""

    def load_text_file(self, file_path: str) -> str:
        """Load text from .txt file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except Exception as e:
            print(f"❌ Error loading text file {file_path}: {e}")
            return ""

    def clean_text(self, text: str) -> str:
        """Clean and preprocess text"""
        # Remove extra whitespace and special characters
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s.,!?;:()\-\[\]{}"\']', ' ', text)
        return text.strip()

    def chunk_text(self, text: str, source: str) -> List[Dict[str, Any]]:
        """Split text into overlapping chunks"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""
        current_length = 0

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            sentence_words = len(sentence.split())

            if current_length + sentence_words <= self.chunk_size:
                current_chunk += sentence + ". "
                current_length += sentence_words
            else:
                if current_chunk:
                    chunks.append({
                        'text': current_chunk.strip(),
                        'source': source,
                        'chunk_id': len(chunks),
                        'word_count': current_length
                    })

                current_chunk = sentence + ". "
                current_length = sentence_words

        if current_chunk:
            chunks.append({
                'text': current_chunk.strip(),
                'source': source,
                'chunk_id': len(chunks),
                'word_count': current_length
            })

        return chunks

    def process_documents(self, file_paths: List[str]) -> List[Dict[str, Any]]:
        """Process multiple documents into chunks"""
        print(f"📚 Processing {len(file_paths)} documents...")

        all_chunks = []

        for file_path in file_paths:
            print(f"📄 Processing: {os.path.basename(file_path)}")

            if file_path.lower().endswith('.pdf'):
                text = self.load_pdf(file_path)
            elif file_path.lower().endswith('.txt'):
                text = self.load_text_file(file_path)
            else:
                print(f"⚠️ Unsupported file type: {file_path}")
                continue

            if not text:
                continue

            clean_text = self.clean_text(text)
            source_name = os.path.basename(file_path)
            chunks = self.chunk_text(clean_text, source_name)
            all_chunks.extend(chunks)

            print(f"   ✅ Created {len(chunks)} chunks")

        print(f"🎉 Total chunks: {len(all_chunks)}")
        return all_chunks

print("✅ DocumentProcessor ready!")
print("🚀 Ready to proceed to Cell 4!")

✅ DocumentProcessor ready!
🚀 Ready to proceed to Cell 4!


In [4]:
class DocumentRetriever:
    """Finds relevant document chunks for questions using AI embeddings"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.documents = []
        print(f"🔍 DocumentRetriever ready with {model_name}")

    def build_index(self, documents: List[Dict[str, Any]]) -> None:
        """Build searchable index from documents"""
        print(f"🏗️ Building search index for {len(documents)} chunks...")

        self.documents = documents
        texts = [doc['text'] for doc in documents]

        # Convert text to AI embeddings (numbers)
        self.embeddings = self.model.encode(texts, show_progress_bar=True)

        # Build fast search index
        dimension = self.embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)

        # Normalize for better similarity search
        faiss.normalize_L2(self.embeddings)
        self.index.add(self.embeddings.astype('float32'))

        print(f"✅ Search index ready! Dimension: {dimension}")

    def retrieve(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """Find most relevant document chunks for a question"""
        if self.index is None:
            raise ValueError("Index not built. Call build_index() first.")

        # Convert question to embedding
        query_embedding = self.model.encode([query])
        faiss.normalize_L2(query_embedding)

        # Search for similar chunks
        scores, indices = self.index.search(query_embedding.astype('float32'), top_k)

        # Return results with similarity scores
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx != -1:
                doc = self.documents[idx].copy()
                doc['similarity_score'] = float(score)
                doc['rank'] = i + 1
                results.append(doc)

        return results

print("✅ DocumentRetriever ready!")
print("🚀 Ready to proceed to Cell 5!")

✅ DocumentRetriever ready!
🚀 Ready to proceed to Cell 5!


In [5]:
class AnswerGenerator:
    """Generates intelligent answers using retrieved context"""

    def __init__(self, model_name: str = "google/flan-t5-base"):
        self.model_name = model_name
        print(f"🤖 Loading answer generation model: {model_name}")

        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.model.to(device)

        # Backup QA pipeline
        self.qa_pipeline = pipeline(
            "question-answering",
            model="distilbert-base-cased-distilled-squad",
            device=0 if torch.cuda.is_available() else -1
        )

        print(f"✅ Answer generator ready!")

    def format_context(self, retrieved_docs: List[Dict[str, Any]]) -> str:
        """Format retrieved documents into context"""
        context_parts = []
        for i, doc in enumerate(retrieved_docs):
            source = doc.get('source', 'Unknown')
            text = doc['text']
            context_parts.append(f"[Document {i+1} - {source}]: {text}")
        return "\n\n".join(context_parts)

    def generate_answer_t5(self, question: str, context: str) -> Dict[str, Any]:
        """Generate answer using T5 model"""
        try:
            # Format input for T5
            input_text = f"Answer the question based on the context.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"

            # Tokenize
            inputs = self.tokenizer(
                input_text,
                max_length=1024,
                truncation=True,
                return_tensors="pt"
            ).to(device)

            # Generate answer
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    max_length=512,
                    num_beams=4,
                    early_stopping=True,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id
                )

            # Decode answer
            answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            return {
                'answer': answer.strip(),
                'confidence': 0.8,  # Default confidence for T5
                'method': 't5'
            }

        except Exception as e:
            return {
                'answer': "Sorry, I encountered an error generating the answer.",
                'confidence': 0.0,
                'method': 't5',
                'error': str(e)
            }

    def generate_answer_qa(self, question: str, context: str) -> Dict[str, Any]:
        """Generate answer using QA pipeline"""
        try:
            if not context:
                return {
                    'answer': "No context available for answering.",
                    'confidence': 0.0,
                    'method': 'qa_pipeline'
                }

            result = self.qa_pipeline(question=question, context=context)

            return {
                'answer': result['answer'],
                'confidence': result['score'],
                'method': 'qa_pipeline'
            }

        except Exception as e:
            return {
                'answer': "Sorry, I couldn't extract an answer.",
                'confidence': 0.0,
                'method': 'qa_pipeline',
                'error': str(e)
            }

    def generate_answer(self, question: str, retrieved_docs: List[Dict[str, Any]],
                       method: str = "t5") -> Dict[str, Any]:
        """Main method to generate answers"""
        if not retrieved_docs:
            return {
                'answer': "No relevant context found to answer the question.",
                'confidence': 0.0,
                'sources': [],
                'method': method
            }

        # Format context
        context = self.format_context(retrieved_docs)

        # Generate answer
        if method == "t5":
            result = self.generate_answer_t5(question, context)
        else:
            result = self.generate_answer_qa(question, context)

        # Add metadata
        sources = [doc.get('source', 'Unknown') for doc in retrieved_docs]
        result.update({
            'sources': sources,
            'context_used': len(retrieved_docs),
            'question': question
        })

        return result

print("✅ AnswerGenerator ready!")
print("🚀 Ready to proceed to Cell 6!")

✅ AnswerGenerator ready!
🚀 Ready to proceed to Cell 6!


In [6]:
class RAGSystem:
    """Complete RAG system combining all components"""

    def __init__(self, chunk_size: int = 300, overlap: int = 50,
                 retrieval_model: str = "all-MiniLM-L6-v2",
                 generation_model: str = "google/flan-t5-base"):

        print("🚀 Initializing RAG System...")

        self.config = {
            'chunk_size': chunk_size,
            'overlap': overlap,
            'retrieval_model': retrieval_model,
            'generation_model': generation_model
        }

        # Initialize components
        self.doc_processor = DocumentProcessor(chunk_size=chunk_size, overlap=overlap)
        self.retriever = DocumentRetriever(model_name=retrieval_model)
        self.generator = AnswerGenerator(model_name=generation_model)

        self.documents = []
        self.is_trained = False

        print("✅ RAG System initialized!")

    def load_documents(self, file_paths: List[str]) -> None:
        """Load and process documents"""
        print("📚 Loading documents into RAG system...")

        # Process documents
        self.documents = self.doc_processor.process_documents(file_paths)

        if not self.documents:
            raise ValueError("No documents were processed!")

        # Build retrieval index
        print("🏗️ Building search index...")
        self.retriever.build_index(self.documents)

        self.is_trained = True
        print(f"🎉 RAG system ready with {len(self.documents)} chunks!")

    def answer_question(self, question: str, top_k: int = 5, method: str = "t5",
                       show_sources: bool = True) -> Dict[str, Any]:
        """Answer a question using the RAG system"""
        if not self.is_trained:
            raise ValueError("System not trained! Load documents first.")

        if not question.strip():
            return {
                'answer': "Please provide a valid question.",
                'confidence': 0.0,
                'error': 'empty_question'
            }

        if show_sources:
            print(f"❓ Question: {question}")
            print(f"🔍 Retrieving top {top_k} relevant documents...")

        # Retrieve relevant documents
        retrieved_docs = self.retriever.retrieve(question, top_k=top_k)

        if show_sources and retrieved_docs:
            print(f"📄 Found {len(retrieved_docs)} relevant documents:")
            for i, doc in enumerate(retrieved_docs, 1):
                score = doc.get('similarity_score', 0.0)
                source = doc.get('source', 'Unknown')
                print(f"   {i}. {source} - Score: {score:.4f}")

        # Generate answer
        if show_sources:
            print(f"🤖 Generating answer using {method} method...")
        result = self.generator.generate_answer(question, retrieved_docs, method=method)

        if show_sources:
            print(f"✅ Answer generated with confidence: {result['confidence']:.3f}")

        return result

    def get_system_stats(self) -> Dict[str, Any]:
        """Get system statistics"""
        if not self.documents:
            return {"status": "No documents loaded"}

        sources = {}
        total_words = 0

        for doc in self.documents:
            source = doc.get('source', 'Unknown')
            word_count = doc.get('word_count', 0)
            sources[source] = sources.get(source, 0) + 1
            total_words += word_count

        return {
            "status": "Ready" if self.is_trained else "Not ready",
            "total_chunks": len(self.documents),
            "total_words": total_words,
            "sources": sources,
            "unique_sources": len(sources),
            "avg_chunk_length": total_words / len(self.documents) if self.documents else 0
        }

print("✅ RAG System ready!")
print("🚀 Ready to proceed to Cell 7!")

✅ RAG System ready!
🚀 Ready to proceed to Cell 7!


In [7]:
from google.colab import files

def upload_documents():
    """Upload documents from your computer"""
    print("📁 UPLOAD YOUR DOCUMENTS")
    print("=" * 30)
    print("Please upload PDF or TXT files")

    uploaded = files.upload()

    uploaded_files = []
    for filename, file_data in uploaded.items():
        with open(filename, 'wb') as f:
            f.write(file_data)
        uploaded_files.append(filename)
        print(f"✅ Uploaded: {filename}")

    return uploaded_files

def create_sample_documents():
    """Create sample documents for testing"""
    print("📝 Creating sample documents...")

    # Sample AI research content
    transformer_content = """
Attention Is All You Need

The Transformer is a neural network architecture based entirely on attention mechanisms.
Unlike recurrent neural networks, Transformers can process sequences in parallel.

Key components include:
- Multi-head self-attention mechanism
- Position encodings
- Feed-forward networks
- Layer normalization

The model achieves state-of-the-art results on machine translation tasks while being more parallelizable than RNNs.
    """.strip()

    rag_content = """
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

RAG combines parametric and non-parametric memory for language generation.
The system consists of:
- A dense passage retriever (DPR)
- A sequence-to-sequence generator (BART)

RAG models can generate more factual and specific responses compared to parametric-only models.
The approach allows updating knowledge without retraining by modifying the retrieval corpus.
    """.strip()

    # Save sample files
    with open('transformer_paper.txt', 'w') as f:
        f.write(transformer_content)

    with open('rag_paper.txt', 'w') as f:
        f.write(rag_content)

    print("✅ Created sample documents:")
    print("   - transformer_paper.txt")
    print("   - rag_paper.txt")

    return ['transformer_paper.txt', 'rag_paper.txt']

print("✅ File upload functions ready!")
print("🚀 Ready to proceed to Cell 8!")

✅ File upload functions ready!
🚀 Ready to proceed to Cell 8!


In [8]:

def quick_setup():
    """Quick setup with default settings"""
    print("⚡ QUICK RAG SETUP")
    print("=" * 20)

    # Create sample documents
    file_paths = create_sample_documents()

    # Initialize RAG system
    rag_system = RAGSystem(
        chunk_size=300,
        retrieval_model='all-MiniLM-L6-v2',
        generation_model='google/flan-t5-base'
    )

    # Load documents
    rag_system.load_documents(file_paths)

    print("🎉 RAG system ready!")
    return rag_system

def setup_with_upload():
    """Setup with your own uploaded files"""
    print("📁 SETUP WITH YOUR FILES")
    print("=" * 25)

    # Upload files
    file_paths = upload_documents()

    if not file_paths:
        print("⚠️ No files uploaded, using samples...")
        file_paths = create_sample_documents()

    # Initialize RAG system
    rag_system = RAGSystem()

    # Load documents
    rag_system.load_documents(file_paths)

    return rag_system

print("✅ Setup functions ready!")
print("🚀 Ready to proceed to Cell 9!")

✅ Setup functions ready!
🚀 Ready to proceed to Cell 9!


In [9]:
def test_rag_system(rag_system):
    """Test the RAG system with sample questions"""
    print("🧪 TESTING RAG SYSTEM")
    print("=" * 25)

    test_questions = [
        "What is the main innovation of the Transformer?",
        "How does RAG work?",
        "What are the key components mentioned?"
    ]

    for i, question in enumerate(test_questions, 1):
        print(f"\n[{i}] Testing: {question}")

        try:
            result = rag_system.answer_question(question, show_sources=False)
            print(f"✅ Answer: {result['answer'][:100]}...")
            print(f"📊 Confidence: {result['confidence']:.3f}")
        except Exception as e:
            print(f"❌ Error: {e}")

    print("\n🎉 Testing complete!")

def system_status(rag_system):
    """Check system status"""
    print("📊 SYSTEM STATUS")
    print("=" * 20)

    if not rag_system.is_trained:
        print("❌ System not ready")
        return

    stats = rag_system.get_system_stats()
    print(f"✅ Status: {stats['status']}")
    print(f"📄 Document chunks: {stats['total_chunks']}")
    print(f"📚 Sources: {stats['unique_sources']}")
    print(f"📝 Total words: {stats['total_words']:,}")

print("✅ Testing functions ready!")
print("🚀 Ready to proceed to Cell 10!")

✅ Testing functions ready!
🚀 Ready to proceed to Cell 10!


In [10]:

def main():
    """Main function to start the RAG system"""
    print("🎯 RAG SYSTEM FOR AI RESEARCH PAPERS")
    print("=" * 40)
    print("Choose your option:")
    print("1. ⚡ Quick setup (sample documents)")
    print("2. 📁 Upload your own files")
    print("3. 📊 Check system status")

    choice = input("\nEnter choice (1-3): ").strip()

    if choice == "1":
        print("\n⚡ Starting quick setup...")
        rag_system = quick_setup()
        return rag_system

    elif choice == "2":
        print("\n📁 Starting setup with file upload...")
        rag_system = setup_with_upload()
        return rag_system

    elif choice == "3":
        print("📊 System status check...")
        # This would check existing system
        return None

    else:
        print("⚠️ Invalid choice, using quick setup...")
        rag_system = quick_setup()
        return rag_system

# Ready to use!
print("🎯 RAG SYSTEM COMPLETE!")
print("=" * 30)
print("🚀 To start: rag_system = main()")

🎯 RAG SYSTEM COMPLETE!
🚀 To start: rag_system = main()


In [11]:
def ask_question_simple(rag_system):
    """Simple interactive question-answering interface"""
    print("\n🤖 RAG QUESTION ANSWERING SYSTEM")
    print("=" * 40)
    print("Ask me questions about your documents!")
    print("Type 'quit', 'exit', or 'stop' to end the session.")
    print("Type 'help' for commands.")
    print("=" * 40)

    # Check if system is ready
    if not rag_system.is_trained:
        print("❌ RAG system not ready! Please load documents first.")
        return

    # Show system status
    stats = rag_system.get_system_stats()
    print(f"📊 System loaded with {stats['total_chunks']} chunks from {stats['unique_sources']} documents")
    print()

    question_count = 0

    while True:
        try:
            # Get user input
            question = input("❓ Your question: ").strip()

            # Handle special commands
            if question.lower() in ['quit', 'exit', 'stop', 'q']:
                print("👋 Thanks for using the RAG system! Goodbye!")
                break

            if question.lower() == 'help':
                print("\n📋 Available commands:")
                print("  - Type your question to get an answer")
                print("  - 'status' - Show system information")
                print("  - 'examples' - Show example questions")
                print("  - 'clear' - Clear screen")
                print("  - 'quit'/'exit'/'stop' - End session")
                print()
                continue

            if question.lower() == 'status':
                print(f"\n📊 System Status:")
                print(f"   Documents: {stats['unique_sources']}")
                print(f"   Chunks: {stats['total_chunks']}")
                print(f"   Total words: {stats['total_words']:,}")
                print(f"   Questions answered: {question_count}")
                print()
                continue

            if question.lower() == 'examples':
                print("\n💡 Example questions you can ask:")
                print("   - What are the main components of the Transformer?")
                print("   - How does attention work?")
                print("   - What is retrieval-augmented generation?")
                print("   - Explain the architecture described in the papers")
                print()
                continue

            if question.lower() == 'clear':
                import os
                os.system('clear' if os.name == 'posix' else 'cls')
                continue

            if not question:
                print("⚠️ Please enter a question.")
                continue

            # Process the question
            print(f"\n🔍 Processing your question...")

            try:
                # Get answer from RAG system
                result = rag_system.answer_question(
                    question,
                    top_k=5,
                    method="t5",
                    show_sources=False
                )

                question_count += 1

                # Display results
                print(f"\n🤖 Answer:")
                print(f"   {result['answer']}")
                print(f"\n📊 Confidence: {result['confidence']:.3f}")

                # Show sources
                if 'sources' in result and result['sources']:
                    print(f"📚 Sources: {', '.join(result['sources'])}")

                print("-" * 50)

            except Exception as e:
                print(f"❌ Error processing question: {e}")
                print("Please try again with a different question.")

        except KeyboardInterrupt:
            print("\n\n👋 Session interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            print("Please try again.")

def ask_question_advanced(rag_system):
    """Advanced interactive interface with more options"""
    print("\n🚀 ADVANCED RAG QUESTION ANSWERING")
    print("=" * 40)

    if not rag_system.is_trained:
        print("❌ RAG system not ready! Please load documents first.")
        return

    # Configuration options
    config = {
        'top_k': 5,
        'method': 't5',
        'show_sources': True,
        'show_retrieved_docs': False
    }

    print("⚙️ Current settings:")
    for key, value in config.items():
        print(f"   {key}: {value}")
    print()

    while True:
        try:
            print("🎯 Options:")
            print("1. Ask a question")
            print("2. Change settings")
            print("3. Show system stats")
            print("4. Exit")

            choice = input("\nSelect option (1-4): ").strip()

            if choice == '1':
                question = input("\n❓ Your question: ").strip()

                if not question:
                    print("⚠️ Please enter a question.")
                    continue

                print(f"\n🔍 Processing with settings: {config}")

                try:
                    result = rag_system.answer_question(
                        question,
                        top_k=config['top_k'],
                        method=config['method'],
                        show_sources=config['show_sources']
                    )

                    print(f"\n🤖 Answer:")
                    print(f"   {result['answer']}")
                    print(f"\n📊 Confidence: {result['confidence']:.3f}")

                    if config['show_retrieved_docs'] and 'sources' in result:
                        print(f"📚 Retrieved from: {', '.join(result['sources'])}")

                    # Ask for feedback
                    feedback = input("\n👍 Was this answer helpful? (y/n): ").strip().lower()
                    if feedback in ['n', 'no']:
                        suggestion = input("💡 How could the answer be improved? ")
                        print(f"📝 Feedback noted: {suggestion}")

                    print("-" * 50)

                except Exception as e:
                    print(f"❌ Error: {e}")

            elif choice == '2':
                print("\n⚙️ Change settings:")
                print("1. Number of documents to retrieve (top_k)")
                print("2. Generation method (t5/qa_pipeline)")
                print("3. Show sources (True/False)")
                print("4. Show retrieved documents (True/False)")

                setting_choice = input("Select setting to change (1-4): ").strip()

                if setting_choice == '1':
                    try:
                        new_k = int(input(f"Current top_k: {config['top_k']}. New value: "))
                        config['top_k'] = max(1, min(50, new_k))
                        print(f"✅ Updated top_k to {config['top_k']}")
                    except ValueError:
                        print("❌ Invalid number.")

                elif setting_choice == '2':
                    method = input("Method (t5/qa_pipeline): ").strip().lower()
                    if method in ['t5', 'qa_pipeline']:
                        config['method'] = method
                        print(f"✅ Updated method to {method}")
                    else:
                        print("❌ Invalid method. Use 't5' or 'qa_pipeline'")

                elif setting_choice == '3':
                    show = input("Show sources (y/n): ").strip().lower()
                    config['show_sources'] = show in ['y', 'yes', 'true']
                    print(f"✅ Updated show_sources to {config['show_sources']}")

                elif setting_choice == '4':
                    show = input("Show retrieved documents (y/n): ").strip().lower()
                    config['show_retrieved_docs'] = show in ['y', 'yes', 'true']
                    print(f"✅ Updated show_retrieved_docs to {config['show_retrieved_docs']}")

            elif choice == '3':
                stats = rag_system.get_system_stats()
                print(f"\n📊 System Statistics:")
                for key, value in stats.items():
                    if isinstance(value, dict):
                        print(f"   {key}:")
                        for k, v in value.items():
                            print(f"     {k}: {v}")
                    else:
                        print(f"   {key}: {value}")

            elif choice == '4':
                print("👋 Goodbye!")
                break

            else:
                print("❌ Invalid option. Please select 1-4.")

        except KeyboardInterrupt:
            print("\n\n👋 Session interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

def batch_question_answering(rag_system, questions_file=None):
    """Process multiple questions from a file or predefined list"""
    print("📋 BATCH QUESTION ANSWERING")
    print("=" * 30)

    if not rag_system.is_trained:
        print("❌ RAG system not ready!")
        return

    # Default questions if no file provided
    default_questions = [
        "What is the main innovation of the Transformer architecture?",
        "How does the attention mechanism work?",
        "What are the key components of a Transformer?",
        "How does RAG combine parametric and non-parametric memory?",
        "What are the advantages of self-attention over recurrent layers?",
        "How does retrieval-augmented generation work?",
        "What is the difference between RAG-Token and RAG-Sequence?",
        "What are the applications of the Transformer model?"
    ]

    questions = default_questions

    if questions_file:
        try:
            with open(questions_file, 'r') as f:
                questions = [line.strip() for line in f if line.strip()]
        except FileNotFoundError:
            print(f"⚠️ File {questions_file} not found. Using default questions.")

    print(f"🔍 Processing {len(questions)} questions...")
    print()

    results = []

    for i, question in enumerate(questions, 1):
        print(f"[{i}/{len(questions)}] {question}")

        try:
            result = rag_system.answer_question(question, show_sources=False)
            results.append({
                'question': question,
                'answer': result['answer'],
                'confidence': result['confidence']
            })

            print(f"✅ Answer: {result['answer'][:100]}...")
            print(f"📊 Confidence: {result['confidence']:.3f}")

        except Exception as e:
            print(f"❌ Error: {e}")
            results.append({
                'question': question,
                'answer': f"Error: {e}",
                'confidence': 0.0
            })

        print("-" * 40)

    # Summary
    avg_confidence = sum(r['confidence'] for r in results) / len(results)
    print(f"\n📈 Summary:")
    print(f"   Questions processed: {len(results)}")
    print(f"   Average confidence: {avg_confidence:.3f}")

    return results

def save_qa_session(results, filename="qa_session.txt"):
    """Save Q&A results to a file"""
    try:
        with open(filename, 'w') as f:
            f.write("RAG Question-Answering Session Results\n")
            f.write("=" * 50 + "\n\n")

            for i, result in enumerate(results, 1):
                f.write(f"Question {i}: {result['question']}\n")
                f.write(f"Answer: {result['answer']}\n")
                f.write(f"Confidence: {result['confidence']:.3f}\n")
                f.write("-" * 30 + "\n\n")

        print(f"✅ Results saved to {filename}")
    except Exception as e:
        print(f"❌ Error saving results: {e}")

print("✅ Interactive interfaces ready!")
print("🚀 Available functions:")
print("   - ask_question_simple(rag_system) - Basic Q&A interface")
print("   - ask_question_advanced(rag_system) - Advanced interface with settings")
print("   - batch_question_answering(rag_system) - Process multiple questions")
print("   - save_qa_session(results) - Save Q&A results to file")

✅ Interactive interfaces ready!
🚀 Available functions:
   - ask_question_simple(rag_system) - Basic Q&A interface
   - ask_question_advanced(rag_system) - Advanced interface with settings
   - batch_question_answering(rag_system) - Process multiple questions
   - save_qa_session(results) - Save Q&A results to file


In [13]:
def run_complete_rag_system():
    """Complete workflow: Setup system and start Q&A"""
    print("🚀 STARTING COMPLETE RAG SYSTEM")
    print("=" * 40)

    # Step 1: Initialize the system
    print("Step 1: Setting up RAG system...")
    rag_system = main()

    if rag_system is None:
        print("❌ System setup failed!")
        return

    # Step 2: Test the system
    print("\nStep 2: Testing the system...")
    test_rag_system(rag_system)

    # Step 3: Show system status
    print("\nStep 3: System status...")
    system_status(rag_system)

    # Step 4: Start interactive Q&A
    print("\nStep 4: Starting interactive question-answering...")

    interface_choice = input("\nChoose interface:\n1. Simple Q&A\n2. Advanced Q&A\n3. Batch processing\nEnter choice (1-3): ").strip()

    if interface_choice == "1":
        ask_question_simple(rag_system)
    elif interface_choice == "2":
        ask_question_advanced(rag_system)
    elif interface_choice == "3":
        results = batch_question_answering(rag_system)
        save_choice = input("\nSave results to file? (y/n): ").strip().lower()
        if save_choice in ['y', 'yes']:
            filename = input("Enter filename (default: qa_session.txt): ").strip()
            if not filename:
                filename = "qa_session.txt"
            save_qa_session(results, filename)
    else:
        print("Starting simple interface by default...")
        ask_question_simple(rag_system)

    return rag_system

In [14]:
print("🎯 COMPLETE RAG SYSTEM READY!")
print("=" * 50)
print("📋 Instructions:")
print("1. Run all cells in order (1-12)")
print("2. Execute: run_complete_rag_system()")
print("3. Follow the prompts to set up and use the system")
print("4. Ask questions using the interactive interface")
print()
print("🚀 To start the complete system:")
print("   run_complete_rag_system()")
print()
print("🎯 Or run individual components:")
print("   rag_system = main()                    # Setup system")
print("   ask_question_simple(rag_system)       # Start Q&A")

# Execute the complete system
if __name__ == "__main__":
    # Uncomment the line below to auto-run the system
    # run_complete_rag_system()
    pass

🎯 COMPLETE RAG SYSTEM READY!
📋 Instructions:
1. Run all cells in order (1-12)
2. Execute: run_complete_rag_system()
3. Follow the prompts to set up and use the system
4. Ask questions using the interactive interface

🚀 To start the complete system:
   run_complete_rag_system()

🎯 Or run individual components:
   rag_system = main()                    # Setup system
   ask_question_simple(rag_system)       # Start Q&A


In [16]:
# Setup system
rag_system = main()

# Start asking questions
ask_question_simple(rag_system)

🎯 RAG SYSTEM FOR AI RESEARCH PAPERS
Choose your option:
1. ⚡ Quick setup (sample documents)
2. 📁 Upload your own files
3. 📊 Check system status

Enter choice (1-3): 1

⚡ Starting quick setup...
⚡ QUICK RAG SETUP
📝 Creating sample documents...
✅ Created sample documents:
   - transformer_paper.txt
   - rag_paper.txt
🚀 Initializing RAG System...
📄 DocumentProcessor ready: 300 words per chunk
🔍 DocumentRetriever ready with all-MiniLM-L6-v2
🤖 Loading answer generation model: google/flan-t5-base


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


✅ Answer generator ready!
✅ RAG System initialized!
📚 Loading documents into RAG system...
📚 Processing 2 documents...
📄 Processing: transformer_paper.txt
   ✅ Created 1 chunks
📄 Processing: rag_paper.txt
   ✅ Created 1 chunks
🎉 Total chunks: 2
🏗️ Building search index...
🏗️ Building search index for 2 chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Search index ready! Dimension: 384
🎉 RAG system ready with 2 chunks!
🎉 RAG system ready!

🤖 RAG QUESTION ANSWERING SYSTEM
Ask me questions about your documents!
Type 'quit', 'exit', or 'stop' to end the session.
Type 'help' for commands.
📊 System loaded with 2 chunks from 2 documents

❓ Your question: 2

🔍 Processing your question...

🤖 Answer:
   [Document 1 - transformer_paper.txt]

📊 Confidence: 0.800
📚 Sources: rag_paper.txt, transformer_paper.txt
--------------------------------------------------
❓ Your question: what is this

🔍 Processing your question...

🤖 Answer:
   Transformer is a neural network architecture

📊 Confidence: 0.800
📚 Sources: rag_paper.txt, transformer_paper.txt
--------------------------------------------------
❓ Your question: exit
👋 Thanks for using the RAG system! Goodbye!


In [17]:
rag_system = main()

# Start asking questions
ask_question_simple(rag_system)

🎯 RAG SYSTEM FOR AI RESEARCH PAPERS
Choose your option:
1. ⚡ Quick setup (sample documents)
2. 📁 Upload your own files
3. 📊 Check system status

Enter choice (1-3): 2

📁 Starting setup with file upload...
📁 SETUP WITH YOUR FILES
📁 UPLOAD YOUR DOCUMENTS
Please upload PDF or TXT files


Saving 1706.03762v7.pdf to 1706.03762v7.pdf
Saving 2005.11401v4.pdf to 2005.11401v4.pdf
Saving 2005.14165v4.pdf to 2005.14165v4.pdf
✅ Uploaded: 1706.03762v7.pdf
✅ Uploaded: 2005.11401v4.pdf
✅ Uploaded: 2005.14165v4.pdf
🚀 Initializing RAG System...
📄 DocumentProcessor ready: 300 words per chunk
🔍 DocumentRetriever ready with all-MiniLM-L6-v2
🤖 Loading answer generation model: google/flan-t5-base


Device set to use cpu


✅ Answer generator ready!
✅ RAG System initialized!
📚 Loading documents into RAG system...
📚 Processing 3 documents...
📄 Processing: 1706.03762v7.pdf
   ✅ Created 22 chunks
📄 Processing: 2005.11401v4.pdf
   ✅ Created 37 chunks
📄 Processing: 2005.14165v4.pdf
   ✅ Created 140 chunks
🎉 Total chunks: 199
🏗️ Building search index...
🏗️ Building search index for 199 chunks...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Search index ready! Dimension: 384
🎉 RAG system ready with 199 chunks!

🤖 RAG QUESTION ANSWERING SYSTEM
Ask me questions about your documents!
Type 'quit', 'exit', or 'stop' to end the session.
Type 'help' for commands.
📊 System loaded with 199 chunks from 3 documents

❓ Your question: what is this

🔍 Processing your question...

🤖 Answer:
   Wikipedia.

📊 Confidence: 0.800
📚 Sources: 2005.14165v4.pdf, 2005.14165v4.pdf, 2005.11401v4.pdf, 2005.11401v4.pdf, 2005.14165v4.pdf
--------------------------------------------------
❓ Your question: exit
👋 Thanks for using the RAG system! Goodbye!
